# Hash function: how does it works?

Blockchain and, in general, distributed ledger technologies are based on the mathematical concept of hashing. An hashing function shows six main characteristics:
- Maps Input x of any size to an Output of fixed size – called a ‘Hash’ 
- Deterministic: Always the same Hash for the same x
- Preimage resistant (One way): infeasible to determine x from Hash(x) 
- Collision resistant: infeasible to find and x and y where Hash(x) = Hash(y) 
- Avalanche effect: Change x slightly and Hash(x) changes significantly 
- Puzzle friendliness: knowing Hash(x) and part of x it is still very hard to find rest of x 

## How can we compute it?
In 2021, every programming language has a library to perform hashing. In python, there are various libraries able to calculate it. In this course, we will use the Ethereum hashing function (keccak256), and we will use the python library maintained by the Ethereum foundation to calculate it.
The name of the library is eth-utils and its github page can be found here, together with the installation instructions: https://github.com/ethereum/eth-utils#crypto-utils
Let's try some examples:

### At first, let's compute the hash of "hello world"

In [2]:
from eth_utils import keccak
keccak(b'Hello world')

b'\xedl\x11\xb0\xb5\xb8\x08\x96\r\xf2o[\xfcG\x1d\x04\xc1\x99[\x0f\xfd U\x92Z\xd1\xbe(\xd6\xba\xad\xfd'

The Keccak256 mapped the 'Hello world' phrase into a 256 bit (32 Bytes) string, here reported in a hexadecimal notation.
The full translation has been:
1. 'Hello world' is codified into a bit string
2. Keccak256 translates the bit string into a 256 bit one, which is called the *hash* of 'Hello world' string.
Now, let's try some of the properties described before:


### Maps Input x of any size to an Output of fixed size – called a ‘Hash’ 

In [8]:
print (keccak(b'Hello world')) #print hello world hash
print (len(keccak(b'Hello world'))) #and its len in hexadecimal code 
print (keccak(b'Some very long phrase just to show how this property works'))
print (len(keccak(b'Some very long phrase just to show how this property works')))
print (keccak(b'A'))
print (len(keccak(b'A')))

b'\xedl\x11\xb0\xb5\xb8\x08\x96\r\xf2o[\xfcG\x1d\x04\xc1\x99[\x0f\xfd U\x92Z\xd1\xbe(\xd6\xba\xad\xfd'
32
b'\xcaL\x88\x10\xf1\xe1\xd5\xec&\xb3\xaf\x80O\xd1\xb3.\x18 \x85w\xe9\xbcl\xa8\x872\x10\xb9\x03\xe6\xd1i'
32
b'\x03x?\xac.\xfe\xd8\xfb\xc9\xadD>Y.\xe3\x0ea\xd6_G\x11@\xc1\x0c\xa1U\xe97\xb45\xb7`'
32


(Here lenght is 32 because are hexadecimal words --->

In [9]:
32*16 #how many bytes?

512

### Deterministic: Always the same Hash for the same x

In [11]:
keccak(b'Hello world') == keccak(b'Hello world') #are the hashes of the same x the same?

True

### Preimage resistant (One way): infeasible to determine x from Hash(x) 
The computational inverse of keccak512 does not exists. The only way to invert it would be to make a random process. Let's give a try.

In [18]:
import secrets #a secure random hex generator for python
import time #The python time standard library
h_w_hash = keccak(b'Hello world')

now = time.time() #save starting time
for i in range(10000): #try 10000 times
    random_number = secrets.token_hex(32) #generate a random 32 digits hexadecimal
    if keccak(random_number.encode('ascii')) == h_w_hash: #if hashes are the same
        print (random_number) #print the hexadecimal
elapsed_time = (time.time() - now) #get elapsed time
print (elapsed_time) #print it

0.18201184272766113


The test of 10'000 strings took 0.18 seconds with my pc. Try with yours. Since we have $2^{512}$ possibilities, on average we will need $\frac{e time \cdot 2^{512}}{10^4}$ seconds to find a solution (which can be anyway different from the original x, since more than one x can give the same hash.

In [25]:
avg_time = int((2**512)*elapsed_time/(10000))
print (avg_time, " seconds;")
print (int(avg_time/3600), " hours;")
print (int(avg_time/3600/24), " days;")
print (int(avg_time/3600/24/365), " years.")

244037982826739988493487585199179670525876422780434911127937718913295195791136091915596976450924256191838561741651970967730644635227535043802972553216  seconds;
67788328562983331073626092569469594381452241860673526251845889602857819777344239428604579650712503835679844323412693870599543656748410663287652352  hours;
2824513690124305345873183268693947283459919071566242617308700592793385191952694117249302993545286699821059925803046490550347195639368284763062272  days;
7738393671573438639822886460107213415012006764977100231600468645346861512204720438395438829160103591981635321334084349869012594416452924080128  years.


Quite a lot.

### Avalanche effect: Change x slightly and Hash(x) changes significantly 
let's hash "Hello world" and "hello world"

In [27]:
print (keccak(b'Hello world'))
print (keccak(b'hello world'))

b'\xedl\x11\xb0\xb5\xb8\x08\x96\r\xf2o[\xfcG\x1d\x04\xc1\x99[\x0f\xfd U\x92Z\xd1\xbe(\xd6\xba\xad\xfd'
b'G\x172\x85\xa8\xd74\x1e^\x97/\xc6w(c\x84\xf8\x02\xf8\xefB\xa5\xec_\x03\xbb\xfa%L\xb0\x1f\xad'


Which are completely different. This means that you cannot try to infer $hash^{-1}(hash(x))$ from $hash^{-1}(hash(z))$, where $z$ is very similar to x 

### Puzzle friendliness: knowing Hash(x) and part of x it is still very hard to find rest of x 
Let's say we know that the hash of a string which starts with "Hello wor" is b'\xedl\x11\xb0\xb5\xb8\x08\x96\r\xf2o[\xfcG\x1d\x04\xc1\x99[\x0f\xfd U\x92Z\xd1\xbe(\xd6\xba\xad\xfd'. 
How can we get the correct hash? we will have to try all the different combinations of it and find a hash which fits. (Which means trying 26*26 letters, assuming we will not include strange symbols, and that the lenght of x is the same of "hello world"). 